In [1]:
! pip install edinet-xbrl

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 

In [62]:
# ファイルパスのパターンにワイルドカードを使うので必要
import glob
# xbrl解析に必要
from edinet_xbrl.edinet_xbrl_parser import EdinetXbrlParser
# 不必要なタグ・スペースを除去するのに必要
import re

# 数字は半角、カナは全角にするのに必要
import unicodedata

# MeCab
import MeCab

#
import pandas as pd

import numpy as np

# xbrlから事業等のリスクの抜き出し

In [6]:
def Business_risk(file_name_list):
    #パーサのインスタンス化
    parser = EdinetXbrlParser()

    for file_name in file_name_list:
        xbrl_path = f"influencer_marketing_data_set/{file_name}/S*/XBRL/PublicDoc/*.xbrl"


        xbrl_path_all = glob.glob(xbrl_path)
        print(xbrl_path_all)

        xbrl_path = glob.glob(xbrl_path)[0]

        edinet_xbrl_object = parser.parse_file(xbrl_path)

        # 事業等のリスクをとってみる
        # <jpcrp_cor:BusinessRisksTextBlock contextRef="FilingDateInstant">
        # ってところに事業等のリスクがあるので
        key='jpcrp_cor:BusinessRisksTextBlock'
        context_ref='FilingDateInstant'
        data = edinet_xbrl_object.get_data_by_context_ref(key, context_ref)
        text_data = data.get_value()

        # 空白（\s）除去
        text_data = re.sub('\s','',text_data)
        # 任意の文字数のタグ（<.*?>）の除去
        text_data = re.sub('<.*?>','',text_data)
        # 2【事業等のリスク】を削除
        text_data = re.sub('^.*?【事業等のリスク】','',text_data)
        # 数字は半角、カナは全角へ
        text_data = unicodedata.normalize("NFKC", text_data)

        # テキストデータの保存
        text_file = open(f'Business_risk/{file_name}_Business_risk.txt', 'w', encoding='UTF-8')
        text_file.write(text_data)
        text_file.close()

<>:25: SyntaxWarning: invalid escape sequence '\s'
<>:25: SyntaxWarning: invalid escape sequence '\s'
C:\Users\rhamano\AppData\Local\Temp\ipykernel_15320\3815388784.py:25: SyntaxWarning: invalid escape sequence '\s'
  text_data = re.sub('\s','',text_data)


In [7]:
file_name_list = ["anymindgroup_2023","cyberbuzz_2023","trenders_2023"]
Business_risk(file_name_list)

['influencer_marketing_data_set/anymindgroup_2023\\S100QHY2\\XBRL\\PublicDoc\\jpcrp030000-asr-001_E37466-000_2022-12-31_01_2023-03-30.xbrl']
['influencer_marketing_data_set/cyberbuzz_2023\\S100SGQG\\XBRL\\PublicDoc\\jpcrp030000-asr-001_E35089-000_2023-09-30_01_2023-12-14.xbrl']
['influencer_marketing_data_set/trenders_2023\\S100R4DE\\XBRL\\PublicDoc\\jpcrp030000-asr-001_E26933-000_2023-03-31_01_2023-06-26.xbrl']


# 事業等のリスクで、Bag-of-Words作成

In [23]:
# テキスト読み込み
with open("Business_risk/anymindgroup_2023_Business_risk.txt", encoding="UTF-8") as f:
    anymindgroup = f.read()

anymindgroup

'本書に記載した事業の状況、経理の状況等に関する事項のうち、投資者の判断に重要な影響を及ぼす可能性のある事項には下記のようなものがあります。なお、これらは、当社グループが事業を遂行する上で発生しうるすべてのリスクを網羅しているものではありません。また、文中における将来に関する事項につきましては別段の記載のない限り、本書提出日現在において当社グループが判断したものであります。(1)事業環境にかかわるリスク1参入市場について(顕在化の可能性:低、顕在化の時期:長期)当社グループが事業を展開しているEC市場、インフルエンサーマーケティング市場、デジタルマーケティング市場、オンライン動画市場では、スマートフォン市場の成長やブロードバンドの普及、新しいテクノロジーの活用により拡大傾向にあります。当社グループはこの成長は継続するものと見込んでおり、現在展開市場を軸に多角的に事業を展開する計画であります。しかしながら、今後国内外の経済情勢や景気動向等の理由により市場成長が鈍化、若しくは市場環境が変化するような場合には、当社グループ財政状態や経営成績に影響を及ぼす可能性があります。2技術革新等について(顕在化の可能性:中、顕在化の時期:長期)当社グループが事業を展開しているEC市場、インフルエンサーマーケティング市場、デジタルマーケティング市場、オンライン動画市場では、技術革新や顧客ニーズの変化のスピードが非常に早く、事業者はその変化に柔軟に対応する必要があります。当社グループにおいても、最新の技術や市場環境の変化を迅速に対応できるよう努めております。しかしながら、当社グループが技術革新や顧客ニーズの変化に対応できない場合、また変化の対応のためのシステムや人件費に多くの投資を要する場合、当社グループの財政状態や経営成績に影響を及ぼす可能性があります。3他社との競合について(顕在化の可能性:中、顕在化の時期:中期)当社グループが事業を展開しているEC市場、インフルエンサーマーケティング市場、デジタルマーケティング市場、オンライン動画市場においては、多くの企業が事業展開しております。当社グループは展開領域において技術力や事業展開力を活かして高付加価値のサービスを提供することで市場における優位性を確立し、競争力を向上させてまいりました。今後もクライアント目線に立ってサービスをより充実させ

In [24]:
# テキスト読み込み
with open("Business_risk/cyberbuzz_2023_Business_risk.txt", encoding="UTF-8") as f:
    cyberbuzz = f.read()

cyberbuzz

'有価証券報告書に記載した事業の状況、経理の状況等に関する事項のうち、経営者が連結会社の財政状態、経営成績及びキャッシュ・フローの状況に重要な影響を与える可能性があると認識している重要なリスクは、以下のとおりであります。なお、当該記載事項は、当連結会計年度末現在において当社グループが判断したものであり、将来において発生の可能性があるすべてのリスクを網羅するものではありません。当社グループはこれらのリスク発生の可能性を認識した上で、発生の回避及び発生した場合の対応に努めてまいります。(1)事業環境に関するリスクについて1業界動向について当社グループは、主にWebメディア及びソーシャルメディアを活用したマーケティング事業を行っております。株式会社電通の「2022年日本の広告費」によれば、2022年の国内インターネット広告市場は、インストリーム広告を中心とした動画広告需要増加や、企業の販売促進活動におけるデジタル活用が進んだことにより、前年比14.3%増の3兆912億円と推計され、2兆円を超えた2019年よりわずか3年で約1兆円増加し、3兆円規模の市場となりました。また、2022年の国内ソーシャルメディアマーケティング市場は、前年比23.0%増の9,317億円、2023年は前年比17.0%増の1兆899億円、2024年は前年比16.7%増の1兆2,713億円と推計されております。今後も堅調に推移すると予想しておりますが、新型コロナウイルス感染症のような外的要因により市場成長が阻害されるような状況が生じた場合、また、広告業界においては、景気変動により広告主の広告支出が増減する傾向があるため、国内マクロ経済の動向及び国内主要産業部門における事業環境が変化した場合には、当社グループの業績及び財政状態に影響を与える可能性があります。2インフルエンサーとの関係当社グループの事業は、クライアント企業のマーケティングに対しサービスを提供しており、その多様なニーズに応えるため、影響力の強いインフルエンサーや、特定分野に特化したインフルエンサーの確保が必要となります。その為、インフルエンサーに対し、クライアント企業の広告案件の継続的なご紹介やSNSへの投稿に関する法令・ガイドラインの遵守等の有用な情報を提供することにより、親密かつ広範なネットワークを構築しております。また、良質なインフルエ

In [25]:
# テキスト読み込み
with open("Business_risk/trenders_2023_Business_risk.txt", encoding="UTF-8") as f:
    trenders = f.read()

trenders

'本書に記載した当社グループの事業の状況及び経理の状況等に関する事項のうち、経営者が連結会社の財政状況、経営成績及びキャッシュ・フローの状況に重要な影響を与える可能性があると認識している主要なリスクは、以下のとおりであります。なお、文中の将来に関する事項は、当連結会計年度末現在において当社グループが判断したものであります。1マーケティング事業の市場動向に係るリスクについて当社グループは、マーケティング事業の美容マーケティング領域において、主にデジタル・SNS領域での企業のプロモーション・PR支援を行っております。日本の広告市場において、当社グループが属するインターネット広告市場は、2022年において前年比14.3%増と大きく成長しております(株式会社電通「2022年日本の広告費」)。今後もインターネット広告市場は中長期的に成長すると予想しておりますが、新型コロナウイルス感染症拡大のような外的要因による経済の停滞及び顧客企業の広告宣伝費の抑制により、短期的に見ると当該市場の成長が鈍化するリスクがあり、当社グループの業績に影響を及ぼす可能性があります。2サービスの陳腐化リスクについてインターネット業界においては、新たな技術やサービスの開発・提供が活発に行われており、常に競合他社より有益な価値を顧客企業に対し提供する必要があります。当社グループでは、顧客企業のニーズに対応するため、常に新たな技術の導入を図り、蓄積したノウハウの活用とあわせてサービス機能の強化及び拡充を進めております。しかしながら何らかの要因により、当社グループが保有するサービス及びノウハウ等が陳腐化し、変化に対する十分な対応が困難となった場合、あるいは変化する顧客企業のニーズに的確な対応ができなくなった場合、当社グループの業績に影響を及ぼす可能性があります。3メディカルマーケティング領域に係るリスクについて当社グループは、マーケティング事業におけるメディカルマーケティング領域において、美容クリニック専売品の開発・販売を行っております。当該領域において、関係する法令や仕入先との契約内容を遵守し、必要な商品管理体制も構築しておりますが、商品に瑕疵等があり当該商品の安全性に問題が生じた場合には、信頼の喪失や損害賠償責任等の発生等により当社グループの業績に影響を及ぼす可能性があります。4インベストメント事業に係る

In [47]:
# テキストデータの単語出現回数を数える
def MeCab_create_bow(text):
    # MeCab用意
    mecabTagger = MeCab.Tagger() # ←（）内で辞書を選択できます

    # テキストの解析結果をnodeとする
    node = mecabTagger.parseToNode(text)

    # output先作成
    noun_count = {} # 辞書型

    while node:
        # 単語抽出
        word = node.surface
        # 品詞抽出
        hinshi = node.feature.split(",")[0] # 1行目：品詞（Pythonは0行目スタートなので0とする）
        # 名詞の単語が、既に検出されている場合
        if word in noun_count.keys() and hinshi == "名詞":
            noun_freq = noun_count[word]
            noun_count[word] = noun_freq + 1 # 既にカウントされている数＋1する
        # 一度も抽出されていない名詞の単語に出会った場合
        elif hinshi == "名詞":
            noun_count[word] = 1 # 用語登録
        # 名詞でない場合はスキップ
        else:
            pass
        # 次の単語へ
        node = node.next

    # 頻出単語順で並べ替え
    noun_count = sorted(noun_count.items(), key=lambda x:x[1], reverse=True)

    # Bag-of-Words作成
    column_name = ["word","count"]
    BoW = pd.DataFrame(noun_count,columns=column_name)
    return BoW

In [48]:
# 形態素解析して、単語の回数をカウント
anymindgroup_bow = MeCab_create_bow(anymindgroup)
cyberbuzz_bow = MeCab_create_bow(cyberbuzz)
trenders_bow = MeCab_create_bow(trenders)

In [50]:
# AnyMind Group株式会社の事業等のリスク

# 単語の種類数
print("単語の種類:" + str(len(anymindgroup_bow)))
# 単語の総出現回数
print("単語の総出現回数:" + str(anymindgroup_bow["count"].sum()))
anymindgroup_bow.head(5)

単語の種類:554
単語の総出現回数:2033


,word,count
0,グループ,74
1,当社,72
2,性,64
3,化,57
4,可能,55


In [51]:
# 株式会社サイバー・バズの事業等のリスク

# 単語の種類数
print("単語の種類:" + str(len(cyberbuzz_bow)))
# 単語の総出現回数
print("単語の総出現回数:" + str(cyberbuzz_bow["count"].sum()))
cyberbuzz_bow.head(5)

単語の種類:518
単語の総出現回数:1642


,word,count
0,等,60
1,当社,59
2,グループ,45
3,性,41
4,事業,36


In [52]:
# トレンダーズ株式会社の事業等のリスク

# 単語の種類数
print("単語の種類:" + str(len(trenders_bow)))
# 単語の総出現回数
print("単語の総出現回数:" + str(trenders_bow["count"].sum()))
trenders_bow.head(5)

単語の種類:347
単語の総出現回数:835


,word,count
0,当社,34
1,グループ,33
2,等,20
3,性,19
4,場合,18


In [37]:
# 一か所にまとめて扱いやすいようにする
bow = pd.merge(anymindgroup_bow, cyberbuzz_bow, on = "word", how="outer")
bow = pd.merge(bow, trenders_bow, on = "word", how="outer")

In [42]:
# カラム書き換えて、NaNを0にしてBoW完成
bow.columns = ["word","anymindgroup_count","cyberbuzz_count","trenders_count"]
bow = bow.fillna(0)
bow.head(5)

,word,anymindgroup_count,cyberbuzz_count,trenders_count
0,グループ,74.0,45.0,33.0
1,当社,72.0,59.0,34.0
2,性,64.0,41.0,19.0
3,化,57.0,13.0,5.0
4,可能,55.0,33.0,17.0


# TF-IDF
- TF-IDF = TF * IDF

## TF表の作成
-TF = 文書d_j中の単語w_iの出現頻度

In [56]:
tf_table = pd.DataFrame()

# TF表作成
tf_table["word"] = bow["word"]
tf_table["anymindgroup"] = bow["anymindgroup_count"] / bow["anymindgroup_count"].sum()
tf_table["cyberbuzz"] = bow["cyberbuzz_count"] / bow["cyberbuzz_count"].sum()
tf_table["trenders"] = bow["trenders_count"] / bow["trenders_count"].sum()

In [57]:
tf_table

,word,anymindgroup,cyberbuzz,trenders
0,グループ,0.036399,0.027406,0.039521
1,当社,0.035416,0.035932,0.040719
2,性,0.031481,0.024970,0.022754
3,化,0.028037,0.007917,0.005988
4,可能,0.027054,0.020097,0.020359
...,...,...,...,...
836,当たり,0.000000,0.000000,0.001198
837,額,0.000000,0.000000,0.001198
838,DOE,0.000000,0.000000,0.001198
839,純資産,0.000000,0.000000,0.001198


## IDFを付け足す
- IDF = log(文書数 / 単語w_iが登場する文書数)
- 単語が使われているのか、使われていないのかが重要なので0,1にtf_tableを返還後にIDFを計算

In [67]:
# 0より大きい要素を1に置き換える
df_numeric = tf_table.drop('word', axis=1).applymap(lambda x: 1 if x > 0 else 0)

# 結果の表示
print(df_numeric.dtypes)
df_numeric

anymindgroup    int64
cyberbuzz       int64
trenders        int64
dtype: object


C:\Users\rhamano\AppData\Local\Temp\ipykernel_15320\617960779.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_numeric = tf_table.drop('word', axis=1).applymap(lambda x: 1 if x > 0 else 0)


,anymindgroup,cyberbuzz,trenders
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,1,1
...,...,...,...
836,0,0,1
837,0,0,1
838,0,0,1
839,0,0,1


In [73]:
# tf表にidf列を追加する
tf_table["idf"] = np.log(3 / (df_numeric["anymindgroup"] + df_numeric["cyberbuzz"] + df_numeric["trenders"]))
# TF-IDF表完成
tf_table

,word,anymindgroup,cyberbuzz,trenders,idf
0,グループ,0.036399,0.027406,0.039521,0.000000
1,当社,0.035416,0.035932,0.040719,0.000000
2,性,0.031481,0.024970,0.022754,0.000000
3,化,0.028037,0.007917,0.005988,0.000000
4,可能,0.027054,0.020097,0.020359,0.000000
...,...,...,...,...,...
836,当たり,0.000000,0.000000,0.001198,1.098612
837,額,0.000000,0.000000,0.001198,1.098612
838,DOE,0.000000,0.000000,0.001198,1.098612
839,純資産,0.000000,0.000000,0.001198,1.098612


In [75]:
# 各TF列に対して、IDF列との席を求める
tf_idf_table = pd.DataFrame()

tf_idf_table["word"] = tf_table["word"]
tf_idf_table["anymindgroup"] = tf_table["anymindgroup"] * tf_table["idf"]
tf_idf_table["cyberbuzz"] = tf_table["cyberbuzz"] * tf_table["idf"]
tf_idf_table["trenders"] = tf_table["trenders"] * tf_table["idf"]

tf_idf_table

,word,anymindgroup,cyberbuzz,trenders
0,グループ,0.0,0.0,0.000000
1,当社,0.0,0.0,0.000000
2,性,0.0,0.0,0.000000
3,化,0.0,0.0,0.000000
4,可能,0.0,0.0,0.000000
...,...,...,...,...
836,当たり,0.0,0.0,0.001316
837,額,0.0,0.0,0.001316
838,DOE,0.0,0.0,0.001316
839,純資産,0.0,0.0,0.001316


In [78]:
# 'word' と 'anymindgroup' 列の選択
anymindgroup_tfidf_ranking = tf_idf_table[['word', 'anymindgroup']]

# 'anymindgroup' 列に基づいて降順にソート
sorted_df = anymindgroup_tfidf_ranking.sort_values(by='anymindgroup', ascending=False)

# 結果の表示
sorted_df.head(10)

,word,anymindgroup
6,:,0.027019
5,顕在,0.009972
22,中期,0.007025
24,中,0.006485
14,時期,0.005385
40,資金,0.003783
44,長期,0.003242
81,在庫,0.002702
23,展開,0.002393
130,使途,0.002162


In [80]:
# 'word' と 'cyberbuzz' 列の選択
cyberbuzz_tfidf_ranking = tf_idf_table[['word', 'cyberbuzz']]

# 'anymindgroup' 列に基づいて降順にソート
sorted_df = cyberbuzz_tfidf_ranking.sort_values(by='cyberbuzz', ascending=False)

# 結果の表示
sorted_df.head(10)

,word,cyberbuzz
554,インフルエンサー,0.008029
558,ガレージ,0.005353
557,投稿,0.005353
559,兆,0.004014
560,プラットフォーム,0.004014
562,審査,0.003345
55,実施,0.003210
571,資本,0.002676
567,億,0.002676
555,株式会社,0.002222


In [81]:
# 'word' と 'trenders' 列の選択
trenders_tfidf_ranking = tf_idf_table[['word', 'trenders']]

# 'anymindgroup' 列に基づいて降順にソート
sorted_df = trenders_tfidf_ranking.sort_values(by='trenders', ascending=False)

# 結果の表示
sorted_df.head(10)

,word,trenders
780,社債,0.003947
779,インベストメント,0.003947
141,顧客,0.002914
788,統制,0.002631
786,償還,0.002631
785,時点,0.002631
784,月末,0.002631
783,メディカル,0.002631
782,ノウハウ,0.002631
781,美容,0.002631
